In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('./data/train.p')
test = pd.read_pickle('./data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return (model, logdir)

In [0]:
def predict(trained_model, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = trained_model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding="same"),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding="same"),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding="same"),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding="same"),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding="same"),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding="same"),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

In [17]:
model = get_cnn_v5(input_shape, num_classes)
(model_trained, logs) = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 11s 41ms/step - loss: 2.4827 - accuracy: 0.2941 - val_loss: 1.1324 - val_accuracy: 0.6411
Epoch 2/5
272/272 [==============================] - 9s 34ms/step - loss: 0.6635 - accuracy: 0.7876 - val_loss: 0.1456 - val_accuracy: 0.9541
Epoch 3/5
272/272 [==============================] - 9s 34ms/step - loss: 0.2351 - accuracy: 0.9295 - val_loss: 0.0433 - val_accuracy: 0.9876
Epoch 4/5
272/272 [==============================] - 9s 34ms/step - loss: 0.1477 - accuracy: 0.9572 - val_loss: 0.0469 - val_accuracy: 0.9854
Epoch 5/5
272/272 [==============================] - 9s 34ms/step - loss: 0.1151 - accuracy: 0.9669 - val_loss: 0.0298 - val_accuracy: 0.9913


0.9662131519274376

In [18]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 4ms/step - loss: 0.1287 - accuracy: 0.9662


0.9662131667137146

In [0]:
def func_obj(params={}):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose=0
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {
      'loss': -accuracy,
      'status': STATUS_OK,
      'model': model
  }

In [12]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.1936 - accuracy: 0.9688

 12/138 [=>............................]
 - ETA: 0s - loss: 0.0786 - accuracy: 0.9740

 23/138 [====>.........................]
 - ETA: 0s - loss: 0.6082 - accuracy: 0.8940

 33/138 [======>.......................]
 - ETA: 0s - loss: 0.4614 - accuracy: 0.9186

 44/138 [========>.....................]
 - ETA: 0s - loss: 0.3814 - accuracy: 0.9261

 55/138 [==========>...................]
 - ETA: 0s - loss: 0.3234 - accuracy: 0.9375


In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding="same"),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding="same"),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding="same"),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj2(params={}):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {
      'loss': -accuracy,
      'status': STATUS_OK,
      'model': model
  }

In [15]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj2,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.1993 - accuracy: 0.9375

 11/138 [=>............................]
 - ETA: 0s - loss: 0.3737 - accuracy: 0.8409

 22/138 [===>..........................]
 - ETA: 0s - loss: 0.5188 - accuracy: 0.8480

 33/138 [======>.......................]
 - ETA: 0s - loss: 0.4472 - accuracy: 0.8759

 44/138 [========>.....................]
 - ETA: 0s - loss: 0.4010 - accuracy: 0.8935

 54/138 [==========>...................]
 - ETA: 0s - loss: 0.4261 - accuracy: 0.8756
